In [ ]:
%tensorflow_version 1.x
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D, Reshape, BatchNormalization, Activation,Convolution2D
from keras.layers import Input, Add, ZeroPadding2D, AveragePooling2D, GlobalMaxPooling2D, add, Concatenate
from keras.datasets import mnist
from keras.datasets import cifar10
import numpy as np
import matplotlib.pyplot as plt
from keras import backend as k
from keras.optimizers import Adam
from PIL import Image as pil_image
from keras.models import Model
from PIL import Image as pil_image
import cv2 as cv2
from  skimage import transform
from keras import optimizers
from keras.datasets import fashion_mnist

TensorFlow 1.x selected.


Using TensorFlow backend.


In [ ]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()
print('Training data Dimension : ', x_train.shape, y_train.shape)
print('Testing data Dimension : ', x_test.shape, y_test.shape)
classes = np.unique(y_train)
number_of_classes = len(classes)
print(number_of_classes)

plt.figure(figsize=[2,2])
plt.imshow(x_train[1000,:,:], cmap='gray')
plt.title("Ground Truth : {}".format(y_train[10]))
print(x_train.shape[1:])

In [ ]:
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
y_train_one_hot = keras.utils.to_categorical(y_train, number_of_classes)
y_test_one_hot = keras.utils.to_categorical(y_test, number_of_classes)

print('Original label : ', y_train[10])
print('After One Hot Encoding : ', y_train_one_hot[10])

In [ ]:
rows, cols = 28,28
ch=1
if k.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], ch, rows, cols)
    x_test = x_test.reshape(x_test.shape[0], ch, rows, cols)
    input_shape = (ch, rows, cols)
else:
    x_train = x_train.reshape(x_train.shape[0], rows, cols, ch)
    x_test = x_test.reshape(x_test.shape[0], rows, cols, ch)
    input_shape = (rows, cols, ch)
print(input_shape)

In [ ]:
def Inception_Module(X,Filter, Name):

  one_one_conv = Conv2D(filters = Filter[0], kernel_size=1, strides=(1,1), padding='same', kernel_initializer=keras.initializers.he_normal(seed=0), activation='relu' , name = Name + 'p_1_1' )(X)

  three_three_conv_step_1 = Conv2D(filters = Filter[1][0], kernel_size = 1, strides = (1,1), padding = 'same', kernel_initializer=keras.initializers.he_normal(seed=0), activation = 'relu', name = Name + 'p_2_1')(X)
  three_three_conv_step_2 = Conv2D(filters = Filter[1][1], kernel_size = 3, strides = (1,1), padding = 'same', kernel_initializer=keras.initializers.he_normal(seed=0), activation = 'relu', name = Name + 'p_2_2')(three_three_conv_step_1)

  five_five_conv_step_1 = Conv2D(filters = Filter[2][0], kernel_size = 1, strides = (1,1), padding = 'same', kernel_initializer=keras.initializers.he_normal(seed=0), activation = 'relu' , name = Name + 'p_3_1')(X)
  five_five_conv_step_2 = Conv2D(filters = Filter[2][1], kernel_size = 5, strides = (1,1), padding = 'same', kernel_initializer=keras.initializers.he_normal(seed=0), activation = 'relu', name = Name + 'p_3_2')(five_five_conv_step_1)

  max_pooling_layer_step_1 = MaxPooling2D(pool_size = (3,3), strides = 1, padding = 'same', name= Name + 'p_4_1')(X)
  max_pooling_layer_step_2 = Conv2D(filters = Filter[3], kernel_size = 1, strides = (1,1), padding = 'same', kernel_initializer=keras.initializers.he_normal(seed=0), activation = 'relu', name= Name + 'p_4_2')(max_pooling_layer_step_1)

  #return Concatenate([one_one_conv, three_three_conv_step_2, five_five_conv_step_2, max_pooling_layer_step_2],axis = 1 name = Name)
  return Concatenate(axis = -1)([one_one_conv, three_three_conv_step_2, five_five_conv_step_2, max_pooling_layer_step_2])

In [ ]:
def GoogLeNet():

  X = Input(input_shape, name='Input')
  first_l = Conv2D(64, kernel_size = 3, padding = 'same', strides = (1,1), kernel_initializer=keras.initializers.he_normal(seed=0), activation = 'relu', name='conv_1st')(X)
  first_l =BatchNormalization(axis=-1)(first_l)

  second_l = Conv2D(192, kernel_size = 3, padding = 'same', strides = (1,1), kernel_initializer=keras.initializers.he_normal(seed=0), activation = 'relu', name='conv_2nd')(first_l)
  second_l =BatchNormalization(axis=-1)(second_l)
  
  three_a = Inception_Module(second_l, [64, (96,128), (16,32), 32], 'three_a_')
  three_a =BatchNormalization(axis=-1)(three_a)

  three_b = Inception_Module(three_a, [128, (128,192), (32,96), 64], 'three_b_')
  three_b =BatchNormalization(axis=-1)(three_b)

  Maxpool_Layer_1 = MaxPooling2D((2,2), strides = (2,2), name = 'Maxpool_After_Stage_3')(three_b)

  four_a =  Inception_Module(Maxpool_Layer_1, [192, (96,208), (16,48), 64], 'four_a_')
  four_a =BatchNormalization(axis=-1)(four_a)

  four_b =  Inception_Module(four_a, [160, (112,224),  (24,64),  64], 'four_b_')
  four_b =BatchNormalization(axis=-1)(four_b)

  four_c =  Inception_Module(four_b, [128, (128,256),  (24,64),  64], 'four_c_')
  four_c =BatchNormalization(axis=-1)(four_c)

  four_d =  Inception_Module(four_c, [192, (96,208), (16,64), 64], 'four_d_')
  four_d =BatchNormalization(axis=-1)(four_d)

  four_e =  Inception_Module(four_d, [256, (160,320), (32,128), 128], 'four_e_')
  four_e =BatchNormalization(axis=-1)(four_e)

  Maxpool_Layer_2 = MaxPooling2D((2,2), strides = (2,2), name = 'Maxpool_After_Stage_4')(four_e)

  five_a =  Inception_Module(Maxpool_Layer_2, [256, (160,320), (32,128), 128], 'five_a_')
  five_a =BatchNormalization(axis=-1)(five_a)

  five_b =  Inception_Module(five_a, [160, (112,224),  (24,64),  64], 'five_b_')
  five_b =BatchNormalization(axis=-1)(five_b)

  Average_pool = AveragePooling2D((7,7), strides = (1,1), padding='valid', name="Average_pooling_Layer")(five_b)


  Flatten_Layer = Flatten()(Average_pool)
  Dropout_Layer = Dropout(0.4, name='Dropout_Layer_1')(Flatten_Layer)
  Dense_layer = Dense(units=256, activation='relu', name='Dense_layer_1')(Dropout_Layer)
  output_Layer = Dense(units=10, activation='softmax', name='output_layer')(Dense_layer)
  
  model = Model(inputs = X, outputs = output_Layer)
  
  return model

In [ ]:
GoogLeNet_Model = GoogLeNet();

Instructions for updating:
If using Keras pass *_constraint arguments to layers.




In [ ]:
GoogLeNet_Model.summary()

In [ ]:
sgd = optimizers.SGD(lr=0.01,decay=1e-6, momentum=0.9, nesterov=True)
GoogLeNet_Model.compile(loss=keras.losses.categorical_crossentropy, optimizer=sgd, metrics=['accuracy'])
batch_size = 256
number_of_epoch = 10

history = GoogLeNet_Model.fit(x_train, y_train_one_hot, validation_split=0.1, epochs=number_of_epoch, batch_size=batch_size, verbose=1, shuffle=1)
report_card = GoogLeNet_Model.evaluate(x_test, y_test_one_hot, verbose=0)
print('Test loss:', report_card[0])
print('Test accuracy:', report_card[1])

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.legend(['loss', 'val_loss'])
plt.title('Loss')

In [ ]:
plt.xlabel('epoch')
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.legend(['accuracy', 'val_accuracy'])
plt.title('Accuracy')
plt.xlabel('epoch')